# Import Libraries

In [3]:
import numpy as np
import pandas as pd
import os
import wave
import glob
import csv
import scipy
import argparse
import pylab
import librosa
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_fscore_support
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import SGD

2022-11-13 10:15:08.084295: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-11-13 10:15:08.084440: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Data-Preprocessing

In [1]:
def mfcc(ans) :
    sample_rate = sr = 16000;
    ans = librosa.feature.mfcc(y=ans, sr=sample_rate, n_fft=2048, 
	                              hop_length=512, n_mfcc=128, window="hann")
    ans = ans / np.max(np.abs(ans))
    return ans

def audio_norm(data):
	'''Normalization of audio'''
	max_data = np.max(data)
	min_data = np.min(data)
	data = (data - min_data)/(max_data - min_data + 1e-6)
	return data - 0.5
    

def eventroll_to_multihot_vector(eventroll):
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    # remove silence class:
    return np.delete(active_events, 8)

def pcen(data, sampling_rate):
	'''Compute Per-Channel Energy Normalization (PCEN)'''
	S = librosa.feature.melspectrogram(data, sr=sampling_rate, power=1)
	log_S = librosa.amplitude_to_db(S, ref=np.max)
	pcen_S = librosa.core.pcen(S)
	return pcen_S

def waveletDenoising(data):
	'''Wavelet Denoising using scikit-image'''
	sigma_est = estimate_sigma(data, multichannel=True, average_sigmas=True)
	im_bayes = denoise_wavelet(data, multichannel=False, convert2ycbcr=True, 
	                           method='BayesShrink', mode='soft')
	im_visushrink = denoise_wavelet(data, multichannel=False, convert2ycbcr=True, 
	                                method='VisuShrink', mode='soft')
	return im_bayes

# Loading Data

In [27]:
tmp = []
test = []
tmp = os.listdir('../input/d/greycollar/mlspassignment2/Y');
tmp.sort();
for i in range(len(tmp)):
    file_name = '../input/d/greycollar/mlspassignment2/Y/' + tmp[i];
    test.append(eventroll_to_multihot_vector(np.load(file_name)));
    test[i] = test[i].flatten()

In [20]:
train = []
tmp = os.listdir('../input/d/greycollar/mlspassignment2/X')
tmp.sort();
for i in range(len(tmp)):
    file_name = '../input/d/greycollar/mlspassignment2/X/' + tmp[i];
    train.append(np.load(file_name));
    # train[i] = np.concatenate((train[i], np.zeros(shape = (1, 64, 250))), axis = 2);
    # train[i] = np.concatenate((train[i], np.zeros(shape = (1, 64, 250))), axis = 2);
    train[i] = train[i].flatten();
#     train[i] = audio_norm(train[i]);
#    train[i] = pcen(train[i], 16000);
#     train[i] = waveletDenoising(train[i]);
#    train[i] = mfcc(train[i]);
#     train[i] = train[i].reshape(train[i].shape[1], train[i].shape[2], 1);

In [6]:
TEST = [];
tmp = os.listdir('../input/test-data/test/X')
tmp.sort();
for i in range(len(tmp)):
    file_name = '../input/test-data/test/X/' + tmp[i];
    TEST.append(np.load(file_name));
    # train[i] = np.concatenate((train[i], np.zeros(shape = (1, 64, 250))), axis = 2);
    # train[i] = np.concatenate((train[i], np.zeros(shape = (1, 64, 250))), axis = 2);
#     train[i] = train[i].flatten();
#     train[i] = audio_norm(train[i]);
#    train[i] = pcen(train[i], 16000);
#     train[i] = waveletDenoising(train[i]);
#    train[i] = mfcc(train[i]);
    TEST[i] = TEST[i].reshape(TEST[i].shape[1], TEST[i].shape[2], 1);

In [21]:
train = np.asarray(train)
train.shape

(10000, 64000)

In [28]:
test = np.asarray(test)
test.shape

(10000, 10)

In [9]:
TEST = np.asarray(TEST)
TEST.shape

(2500, 64, 1000, 1)

In [10]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2022-11-13 10:18:09.865123: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-13 10:18:09.869143: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-11-13 10:18:09.869182: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-13 10:18:09.869205: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (13d1f306d0b2): /proc/driver/nvidia/version does not exist
2022-11-13 10:18:09.873495: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

REPLICAS:  8


# Data Split

In [11]:
# train_x, test_x, train_y, test_y = train_test_split(train, test, test_size = 0.20, random_state = 0)
# del(test)
# del(train)

In [12]:
def details(true, pred):
  result = precision_recall_fscore_support(true, pred, average=None)
  print(print("Precision -> ", result[0].mean() * 100, "\nRecall -> ", result[1].mean() * 100, "\nF1 Score -> ", result[2].mean() * 100));

In [13]:
def converter(pred):
    prediction = pd.DataFrame(columns=['file_name', 'prediction'])
    event_types = ['Alarm_bell_ringing', 'Blender', 'Cat', 'Dishes', 'Dog', 'Electric_shaver_toothbrush', 'Frying', 'Running_water', 'Speech', 'Vacuum_cleaner']
    for i in range(pred.shape[0]):
        event = [];
        for j in range(10):
            if pred[i][j] == 1:
                event.append(event_types[j]);
        event = np.asarray(event, dtype="object")
        prediction.loc[i] = [tmp[i], event];
    return prediction;        

# CNN

In [14]:
## CNN Model

# create model
with strategy.scope():
    model = Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(train[0].shape[0], train[0].shape[1], train[0].shape[2])))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='sigmoid'))

    # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# CNN Model Fit
model.fit(train, test, validation_split=0.2, epochs=25, batch_size=64, verbose=1)

2022-11-13 10:18:17.055463: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2048000000 exceeds 10% of free system memory.


Epoch 1/25
125/125 [==============================] - ETA: 0s - loss: 0.4749 - accuracy: 0.1105

2022-11-13 10:18:50.543255: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 512000000 exceeds 10% of free system memory.


125/125 [==============================] - 56s 285ms/step - loss: 0.4742 - accuracy: 0.1105 - val_loss: 0.4647 - val_accuracy: 0.1195
Epoch 2/25
125/125 [==============================] - 6s 51ms/step - loss: 0.3222 - accuracy: 0.1167 - val_loss: 0.4122 - val_accuracy: 0.1195
Epoch 3/25
125/125 [==============================] - 6s 48ms/step - loss: 0.3001 - accuracy: 0.1083 - val_loss: 0.3985 - val_accuracy: 0.1200
Epoch 4/25
125/125 [==============================] - 6s 49ms/step - loss: 0.2756 - accuracy: 0.1110 - val_loss: 0.3320 - val_accuracy: 0.1360
Epoch 5/25
125/125 [==============================] - 6s 48ms/step - loss: 0.2437 - accuracy: 0.1296 - val_loss: 0.2984 - val_accuracy: 0.1370
Epoch 6/25
125/125 [==============================] - 6s 48ms/step - loss: 0.2110 - accuracy: 0.1513 - val_loss: 0.2876 - val_accuracy: 0.1785
Epoch 7/25
125/125 [==============================] - 6s 51ms/step - loss: 0.1774 - accuracy: 0.1742 - val_loss: 0.2942 - val_accuracy: 0.1445
Epoch 8/

In [16]:
pred_cnn = model.predict(TEST)
for i in range(pred_cnn.shape[0]):
    for j in range(10):
        if pred_cnn[i][j] >= 0.5:
            pred_cnn[i][j] = 1;
        else : 
            pred_cnn[i][j] = 0;
            
pred_cnn

2022-11-13 10:21:48.519872: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 640000000 exceeds 10% of free system memory.


array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [18]:
# details(test_y, pred_cnn)
# print(classification_report(test_y,pred_cnn))

In [19]:
prediction = converter(pred_cnn)
prediction.to_csv('prediction.csv')

/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# DNN

In [29]:
## DNN Model

## create model 
model = Sequential()
model.add(Dense(128, input_shape=(64000, ), activation='relu', kernel_regularizer='l2'))
model.add(Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='sigmoid'))


# complie model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
## DNN Model Fit
model.fit(train, test, validation_split=0.2, epochs=20, batch_size=64, verbose=1)

Epoch 1/20
125/125 [==============================] - 5s 39ms/step - loss: 1.0713 - accuracy: 0.1078 - val_loss: 0.4767 - val_accuracy: 0.1195
Epoch 2/20
125/125 [==============================] - 4s 29ms/step - loss: 0.4799 - accuracy: 0.1128 - val_loss: 0.4417 - val_accuracy: 0.1195
Epoch 3/20
125/125 [==============================] - 4s 29ms/step - loss: 0.4538 - accuracy: 0.1091 - val_loss: 0.4163 - val_accuracy: 0.1195
Epoch 4/20
125/125 [==============================] - 4s 29ms/step - loss: 0.4075 - accuracy: 0.1106 - val_loss: 0.4022 - val_accuracy: 0.1195
Epoch 5/20
125/125 [==============================] - 4s 29ms/step - loss: 0.4048 - accuracy: 0.1095 - val_loss: 0.3992 - val_accuracy: 0.1195
Epoch 6/20
125/125 [==============================] - 4s 29ms/step - loss: 0.4092 - accuracy: 0.1091 - val_loss: 0.3997 - val_accuracy: 0.1195
Epoch 7/20
125/125 [==============================] - 4s 30ms/step - loss: 0.4023 - accuracy: 0.1059 - val_loss: 0.4219 - val_accuracy: 0.1195

In [34]:
## DNN Predict
pred_dnn = model.predict(TEST)
for i in range(len(pred_dnn)):
    for j in range(10):
        if pred_dnn[i][j] >= 0.5:
            pred_dnn[i][j] = 1;
        else :
            pred_dnn[i][j] = 0;

In [35]:
# details(test_y, pred_dnn)
# print(classification_report(test_y,pred_dnn))

# SVM

In [ ]:
## Model Training
svm = OneVsRestClassifier(SVC(max_iter=20)).fit(train_x, train_y)

In [ ]:
pred_svm = svm.predict(test_x);
for i in range(len(pred_svm)):
    for j in range(10):
        if pred_svm[i][j] >= 0.5:
            pred_svm[i][j] = 1;
        else :
            pred_svm[i][j] = 0;

pred_svm

In [ ]:
details(test_y, pred_svm)
print(classification_report(test_y,pred_svm))

# RNN

In [ ]:
## RNN Model
with strategy.scope():
    model = Sequential()
    model.add(LSTM(256, input_shape=(64, 1000), activation='relu', return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='sigmoid'))
    
    # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#RNN Model Fit
model.fit(train_x, train_y, validation_split=0.2, epochs=10, batch_size=64, verbose=1)

In [ ]:
pred_rnn = model.predict(test_x);
for i in range(len(pred_rnn)):
    for j in range(10):
        if pred_rnn[i][j] >= 0.5:
            pred_rnn[i][j] = 1;
        else :
            pred_rnn[i][j] = 0;

pred_rnn

In [ ]:
details(test_y, pred_rnn)

# GMM

In [ ]:
## GMM Model
gmm = GaussianMixture(n_components=3).fit(train_x)

In [ ]:
pred_gmm = gmm.predict(test_x)
for i in range(len(pred_gmm)):
    for j in range(10):
        if pred_gmm[i][j] >= 0.5:
            pred_gmm[i][j] = 1;
        else :
            pred_gmm[i][j] = 0;

pred_gmm

In [ ]:
details(test_y, pred_gmm)
print(classification_report(test_y,pred_gmm))

# RCNN

In [ ]:
with strategy.scope():
    model =  models.Sequential([   
              layers.Conv2D(64 , (3,3),activation = 'relu',input_shape=(train_x[0].shape[0], train_x[0].shape[1], train_x[0].shape[2])),  
              layers.MaxPooling2D(2, padding='same'),
              layers.BatchNormalization(),
              layers.Dropout(0.5),
              layers.Conv2D(64, (3,3), activation='relu'),
              layers.MaxPooling2D(2, padding='same'),
              layers.BatchNormalization(),
              layers.Dropout(0.5),
              layers.Conv2D(64, (3,3), activation='relu'),
              layers.MaxPooling2D(2, padding='same'),
              layers.Flatten(),
              layers.Reshape((27776, 2)),
              layers.LSTM(4),
              layers.Dense(64 , activation = 'relu'),
              layers.Dense(10 , activation = 'sigmoid')
    ])
    
    # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#RCNN Model Fit
model.fit(train_x, train_y, validation_split=0.2, epochs=10, batch_size=64, verbose=1)

In [ ]:
pred_rcnn = model.predict(test_x);
for i in range(len(pred_rcnn)):
    for j in range(10):
        if pred_rcnn[i][j] >= 0.5:
            pred_rcnn[i][j] = 1;
        else :
            pred_rcnn[i][j] = 0;

pred_rcnn

In [ ]:
details(test_y, pred_rcnn)
print(classification_report(test_y,pred_rcnn))

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(train_x, train_y)

In [ ]:
pred_knn = knn.predict(test_x);
for i in range(len(pred_knn)):
    for j in range(10):
        if pred_knn[i][j] >= 0.5:
            pred_knn[i][j] = 1;
        else :
            pred_knn[i][j] = 0;

pred_knn

In [ ]:
details(test_y, pred_knn);
print(classification_report(test_y,pred_knn))